# MULTEM model creation guide
This notebook serves as a guide to making models for multislice image simulations using MULTEM and mul2py. It covers some of the basic requirements of models from a multislice point of view, as well as some technical points.

## Relationship between model size, resolution, and scattering angles
When making models for multislice image simulations you must decide on a suitable slab size. It is important to remember that the $x$ and $y$ size of this model will affect the resolution of the simulation and the maximum possible scattering angle. In other words, the model must be made to fit your requirements for the simulation. As an example, if you perform a STEM simulation with collection semi-angles outside the actual scattering range of your simulation, you will only get black pixels!

For multislice simulations, the number of beams is given by the number of potential pixels $n_x$, and the model size $L_x$ determines the smallest scattering vector $k_x^{min}=1/L_x$. The maximum scattering vector is then $k_x^{max} = n_x * k_x^{min} = n_x / L_x$. In addition, to remove aliasing effects, a bandwidth limit of $2/3$ is often used, so the actual maximum scattering vector is $k_x^{eff} = k_x^{max} * 2/3 = \frac{2n_x}{3L_x}$. Now, $k_x^{eff}$ here is not actually the wave vector transfer between the incoming beam and a scattered beam. It is the largest possible scattering vector, which is actually the transfer between to scattered beams $g_+$ and $g_-$; $k_x^{eff} = g_{+} - g_{-}$, where $g$ is the wave vector of the beam scattered to the highest allowed angle, and the subscript indicates the "sign" of the angle. The simulations are symmetrical in the sense that $g_{+} = - g_{-}$, so that $k_x^{eff}=2g^{max}$, where $g^{max}=|g_{+}|=|g_{-}|$. Simultaneusly, the scattering angle of the beam with $g=g^{max}$ is $\theta^{max}=\lambda g^{max}$. This means that the maximum effective scattering angle of your simulation will be

$$\theta^{max} = \lambda g^{max} = \lambda \frac{1}{2}k^{eff} = \lambda \frac{n_x}{3L_x}$$

if a bandwidth of $2/3$ are used. With no bandwidth, this angle is increased by $3/2$, but this is not recommended, as aliasing effects will appear in the simulations. If you need a higher maximum scattering angle, you should instead increase the potential sampling (will increase simulation time) or reduce the model size (keeps simulation time constant). Changing either of these will change the resolution of your simulation however!

#### A note on convergence angles
You might also want to consider the relationship between the convergence angle you use and the model size if you use very thick models. Because multislice simulations are periodic in $x$ and $y$, a beam that exits the model on one side will enter it on the opposite side. This means, that if you have a thick model, your incident beam might interact with itself if the convergence angle is too large and the model $x$ and $y$ dimensions are too small. The beam radius $r$ should be smaller than $L_x/2$ and $L_y/2$, and for a convergence semi-angle of $\alpha$, and a model thickness of $L_z$, we get

$$ \alpha = \tan\left( \frac{r}{L_z} \right) \Rightarrow r = L_z \tan^{-1}\left(\alpha\right) $$,

and 

$$ r\lt\frac{L}{2} \Rightarrow L_z \lt \frac{L}{2 \tan^{-1}\left(\alpha\right)} $$, 

with $L$ being the smaller of $L_x$ and $L_y$.

With $\alpha=27$ mrad, we get that the maximum model thickness for a model measuring $40.5$ Å in $x$ and $y$ directions is $ L_z^{max} = \frac{40.5\,Å}{2\tan^{-1}\left(\frac{27}{1000}\right)} = 750$ Å, so we see that this is rarely a problem for realistic model sizes, but it should still be considered.

### Example
Let us make a model for a HAADF STEM image simulation. If we want to use a maximum collection semi-angle up to $200$ mrad, which is a commonly used collection angle for atomically resolved images for this material, you will need a model size of

$$L_x = \frac{n_x\lambda}{3\theta^{max}} = \frac{n_x\lambda}{3 \cdot 0.2\,rad} $$

If you want to use $n_x=1024$ pixels and electrons with $\lambda = 0.0251$ Å (corresponding to $200$ kV), you will need a model size that is at least

$$L_x \leq \frac{1024 \cdot 0.0251\,Å}{3 \cdot 0.2\,rad} = 42.84 $$ Å.

Any larger model will result in a too fine resolution of scattering angles and too few pixels, making it impossible for some of the outer ranges of the annual detector to be illuminated.

Now, lets see how many Aluminium unit cells we need to make a HAADF STEM simulation with collection angles at least up to $200$ mrad. Aluminium has a lattice constants $a=b=c=4.05$ Å, so for a model with $L\leq42.84$ Å we will need a model with $$n_a = n_b = \left\lfloor \frac{L}{a} \right\rfloor = \left\lfloor \frac{42.84 Å}{4.05 Å} \right\rfloor = \left\lfloor 10.58 \right\rfloor = 10$$ unit cells. This gives a model with $L_x=L_y = 40.5$ Å, and a maximum scattering angle of $\theta^{max}=\frac{1024 \cdot 0.0251\,Å}{3\cdot40.5\,Å}=0.211=211$ mrad. This means that we can use collection semi-angles up to $211$ mrad with an Al model with $10\times10$ unit cells when we resolve the crystal potential with 1024 pixels and use $200$ kV electrons. Now, we can make our model with ASE.

## Making models with ASE
Making models with ASE is relatively easy. One way is to make the model completely in ASE from scratch. The way we will be using in this guide is to use a premade crystal information file (.cif) that was made in VESTA. First, we load the unit cell file "Al.cif" and then weduplicate this to make a suitable slab. 

In [1]:
from ase.io import read
from pathlib import Path
Al = read(Path(r'C:\Users\emilc\Desktop\Al_2.cif'), reader='ase') #Load the crystal 
#Al = aio.read(Path(r'Al.cif'), store_tags=True) #Load the crystalinformation file
print(Al.info)
na, nb, nc = 10, 10, 20 # Number of unit cells along a, b, and c crystal axes

slab = Al*[na, nb, nc] #Duplicate the model to make a slab

slab.center(axis=(0, 1)) #Center the slab in the x-y plane - leave it unchanged in z-direction


TypeError: read_cif() got an unexpected keyword argument 'reader'

In [81]:
import ase.io as aio
i=0
with open(Path(r'C:\Users\emilc\Desktop\Al_2.cif'), 'r') as ciffile:
    atom = aio.cif.read_cif(ciffile, index=i, store_tags=True)
    for a in atom:
        print(a)
    i += 1

IndexError: list index out of range

Now we have a model with some atoms. To visualize the model in 3D, use ASEs view function:

In [5]:
from ase.visualize import view
view(slab)


If you want to store this model as a .cif file for later reference, you can write the model to a new file like this:

In [3]:
from ase.io import write
write('Al_10x10x20.cif', slab)

## Making models for MULTEM
Now we need to make this model useful for MULTEM. There are many ways of doing this, but the core concept is to make a MATLAB .mat file with some specific field names and values:
  - `spec_atoms`: an array with atomic numbers, positions, and rms3d values, see below for the format.
  - `spec_lx`: Model size in $x$ in Å
  - `spec_ly`: Model size in $y$ in Å
  - `spec_lz`: Model size in $z$ in Å
  - `spec_dz`: Slice thickness in Å
  - `a`: unit cell parameter $a$, optional (required when using the mul2py setup functions)
  - `b`: unit cell parameter $b$, optional (required when using the mul2py setup functions)
  - `c`: unit cell parameter $c$, optional (required when using the mul2py setup functions)
  - `na`: number of unit cells in $a$, optional
  - `nb`: number of unit cells in $b$, optional
  - `nc`: number of unit cells in $c$, optional

The `rms3d` values are related to the Debye-Waller factors $B$ by $rms_{3D} = \sqrt{B / 8  \pi^2}$. The Debye-Waller factors themselves depend on the atomic species (really the crystal - but it is common to take the values for single elements from pure crystals) and the temperature $T$. There are many sources for these values, but one which is commonly used is [Peng et al.](http://scripts.iucr.org/cgi-bin/paper?S010876739600089X). Extensive tables can be found in the [supplementary material](http://scripts.iucr.org/cgi-bin/sendsupfiles?zh0008&file=zh0008_82472sup1.pdf&mime=application/pdf) to this work.

The `spec_atoms` object for a single Al unit cell with $T=300$ K would be an array looking like this (except the header):

| $Z$ | $x$ [Å] | $y$ [Å] | $z$ [Å] | rms3d [Å] | Occupancy | Label | Charge [C] |
|-----|---------|---------|---------|-----------|-----------|-------|------------|
| 13  | 0       | 0       | 0       | 0.1006    | 1         | 0     | 0          |
| 13  | 2.025   | 2.025   | 0       | 0.1006    | 1         | 0     | 0          |
| 13  | 0       | 2.025   | 2.025   | 0.1006    | 1         | 0     | 0          |
| 13  | 2.025   | 0       | 2.025   | 0.1006    | 1         | 0     | 0          |

To help write these values into a .mat file, `mul2py` provides several functions for both writing and converting files to .mat format.

## Saving models with mul2py
With mul2py, you can use `mul2py.io.save_multem_model()`to either convert a predefined .cif file to a .mat file, or to store an ASE object to a .mat file directly. This function takes three required parameters: the name of the .mat file, the model to be written (either a path or an ASE object), and the rms3d values (the Debye-Waller factors can be given directly instead as well). The rms3d values and Debye-Waller factors are not provided as positional arguments, but rather as keyword arguments and may therfore appear to be optional. They are not however.

In addition, you can specify the slice thickness you want to use directly, specify the number of slices, or you can let the function make an educated guess (based on model dimensions and crystal unit cell parameters). The last option is not advised, as you should really have control over this important parameter yourself.

Let us try to write our Al model to a .mat file:

In [2]:
import mul2py as m2p
dwfs = {13: 0.1006}
a, b, c, alpha, beta, gamma = Al.get_cell_lengths_and_angles()
dz = c/2
m2p.io.save_multem_model('Al_10x10x20.mat', slab, dz=dz, B=dwfs, a=a, b=b, c=c, na=na, nb=nb, nc=nc)

Alternatively, you could convert the cif file we made previously (remember to specify number of slices or the slice thickness!):

In [5]:
import mul2py as m2p
dwfs = {13: 0.1006}
dz = 2.025
m2p.io.save_multem_model('Al_10x10x20.mat', 'Al_10x10x20.cif', B=dwfs, dz=dz)

`m2p.io.save_multem_model()` only operates with single Debye-Waller factors or RMS3D values for a given species. If you need to store crystals where atoms of the same species but at different sites have differnt Debye-Waller factors, things become a little bit more complicated when converting CIF files because ASE does not read this information from the ".cif" files. Instead, you can use `m2p.io.cif.convert_cif2mat()` to directly convert a CIF file without loading it into ASE, and the Debye-Waller factors of individual sites are kept. This function requires that the unit cell or crystal in the .cif file was stored with the correct orientation relative to X, Y, and Z. It supports replication, so that you may store a single unit cell in the .cif file and then replicate it to a larger slab on the fly before converting it. You must, however, still supply a slice thickness:

In [1]:
import mul2py as m2p
dz = 2.025
m2p.io.convert_cif2mat('Al.cif', dz=dz, na=10, nb=10, nc=30)

C:\Users\emilc\Documents\PhDScripts\TEMGeminiCentre\mul2py\mul2py\io\cif.py:62: UserWarning: String site labels may not be supported by MULTEM
  warn('String site labels may not be supported by MULTEM')


Converting labels to integers
Saving crystal
Saved crystal to "Al_10x10x30.mat"


You are now ready to set up a MULTEM simulation using one of `mul2py`s MATLAB setup functions!

## Getting your model back from .mat
Sometimes, you would want to retrieve your model from the .mat format and make it back into a .cif file. `mul2py` currently provides a very rough function for doing this (which was made in a hurry - so don't expect too much):

In [6]:
import mul2py as m2p
model = m2p.io.mat2cif('Al_10x10x20.mat')

This function converts the model into a .cif file, and returns the model as an ASE object.